In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt
import kalepy as kale


import matplotlib.cm as cm
from holodeck import plot, utils
from holodeck.constants import YR, MSOL, GYR, PC, MPC

# Build Semi-Analytic Model

In [ ]:
dur, cad = 17.5*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model()
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version

## Test GWB calculation with default Hard_GW

In [ ]:
# just using gwb() function
gwb = sam.gwb_old(fobs_gw_edges, hard=holo.hardening.Hard_GW, realize=10)
fig = plot.plot_gwb(fobs_gw_cents, gwb)

## Test GWB calculation with Fixed_Time model

In [ ]:
# just using gwb() function

gwb = sam.new_gwb(fobs_gw_edges, hard=holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR), realize=10)
fig = plot.plot_gwb(fobs_gw_cents, gwb)

# Calculate SS and GWB strain with sam.gwb()
This function calls single_sources.ss_gws_redz()
## using Hard_GW
### with params

In [ ]:
hard=holo.hardening.Hard_GW()
assert isinstance(hard, (holo.hardening.Fixed_Time_2PL_SAM, holo.hardening.Hard_GW))

In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_gw_edges, hard=holo.hardening.Hard_GW(), realize = 10, loudest = 5, params = True)
print(bgpar.shape, sspar.shape)

In [ ]:
fig = plot.plot_bg_ss(fobs_gw_cents, hc_bg, hc_ss)

In [ ]:
fig = plot.plot_pars(fobs_gw_cents, sspar, bgpar)

### without params

In [ ]:
hc_ss, hc_bg = sam.gwb(fobs_gw_edges, hard=holo.hardening.Hard_GW(), realize=30, loudest=5, params=False)
fig = plot.plot_bg_ss(fobs_gw_cents, hc_bg, hc_ss)

## using Fixed_Time
### with params

In [ ]:
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_gw_edges, hard=hard, realize = 10, loudest = 5, params = True)
fig = plot.plot_bg_ss(fobs_gw_cents, hc_bg, hc_ss)
fig = plot.plot_pars(fobs_gw_cents, sspar, bgpar )

### without params

In [ ]:
hc_ss, hc_bg = sam.gwb(fobs_gw_edges, hard=hard, realize = 10, loudest = 5, params = False)
fig = plot.plot_bg_ss(fobs_gw_cents, hc_bg, hc_ss)

# Plot Initial Redshifts and Pair Fractions

In [ ]:
fig = plot.plot_bg_ss(fobs_gw_cents, bgpar[2,...], sspar[2,...])
ax = fig.axes[0]
ax.set_ylabel('Initial Redshift, $z$')


In [ ]:
def P_of_z(z):
    return (0.03*(1+z)*0.8)

fig = plot.plot_bg_ss(fobs_gw_cents, P_of_z(bgpar[2,...]), P_of_z(sspar[2,...]))
ax = fig.axes[0]
ax.set_ylabel('Pair Fraction, $P(z)$')

# Plot New BG Pars

In [ ]:
print(bgpar.shape)
par_names = holo.single_sources.par_names
print(par_names)

print(sspar.shape)
sspar_all = holo.single_sources.all_sspars(fobs_gw_cents, sspar)
print(sspar_all.shape)

In [ ]:
print(sam.mrat)

In [ ]:
labels = holo.single_sources.par_labels
units = holo.single_sources.par_units

for ii in range(len(bgpar)):
    fig = plot.plot_bg_ss(fobs_gw_cents, bgpar[ii,...]*units[ii], sspar_all[ii,...]*units[ii], ylabel=labels[ii])

Get rest of single source params for comparison